In [9]:
import os
import logging
import pandas as pd
import pandas_gbq
from google.cloud import bigquery
from google.oauth2 import service_account

In [10]:
# Configurar logging
logging.basicConfig(level=logging.INFO)

# Configurar chave de autenticação
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"C:\Users\marcell.oliveira\Desktop\Chinook\sincere-actor-393814-8209ed1b42a4.json"
# Definir credenciais 
credentials = service_account.Credentials.from_service_account_file(r"C:\Users\marcell.oliveira\Desktop\Chinook\sincere-actor-393814-8209ed1b42a4.json")
# Conectar ao BigQuery
client = bigquery.Client()



In [ ]:
# Carregar dados da camada bronze
logging.info("Carregando dados da camada Silver...")
query = "SELECT * FROM camada_silver.Customer"
df_Customer_silver = client.query(query).to_dataframe()

In [ ]:
# Nome do projeto, dataset e tabela
project_id = "sincere-actor-393814"
dataset_id = "camada_gold"
table_id = "dVendedor"

# Definir o schema com `NOT NULL` e descrição
schema = [
    bigquery.SchemaField("id_vendedor", "INTEGER", mode="REQUIRED", description="Identificador único do vendedor"),
    bigquery.SchemaField("nome_completo", "STRING", mode="NULLABLE", description="Nome completo do vendedor"),
    bigquery.SchemaField("empresa", "STRING", mode="NULLABLE", description="Nome da empresa associada ao vendedor"),
    bigquery.SchemaField("endereco", "STRING", mode="NULLABLE", description="Endereço do vendedor"),
    bigquery.SchemaField("cidade", "STRING", mode="NULLABLE", description="Cidade do vendedor"),
    bigquery.SchemaField("uf", "STRING", mode="NULLABLE", description="Estado (UF) do vendedor"),
    bigquery.SchemaField("pais", "STRING", mode="NULLABLE", description="País do vendedor"),
    bigquery.SchemaField("cep", "STRING", mode="NULLABLE", description="Código postal (CEP) do vendedor"),
    bigquery.SchemaField("telefone", "STRING", mode="NULLABLE", description="Número de telefone do vendedor"),
    bigquery.SchemaField("fax", "STRING", mode="NULLABLE", description="Número de fax do vendedor"),
    bigquery.SchemaField("email", "STRING", mode="NULLABLE", description="Endereço de email do vendedor"),
    bigquery.SchemaField("loja", "INTEGER", mode="NULLABLE", description="Identificador da loja associada ao vendedor"),
]

# Configurar a referência para a tabela com clustering
table_ref = client.dataset(dataset_id).table(table_id)
table = bigquery.Table(table_ref, schema=schema)
table.clustering_fields = ["id_vendedor"]  # Definir o campo de clustering

# Criar a tabela com o schema e clustering
try:
    table = client.create_table(table, exists_ok=True)
    logging.info("Tabela criada com sucesso com clustering e campo `id_vendedor` como REQUIRED.")
except Exception as e:
    logging.error(f"Erro ao criar a tabela: {e}")

# Inserir dados na tabela a partir da consulta SQL
sql = f"""
INSERT INTO `{project_id}.{dataset_id}.{table_id}`
SELECT 
    DISTINCT 
    CAST(CustomerId AS INT64) AS id_vendedor,
    CAST(CONCAT(FirstName, ' ', LastName) AS STRING) AS nome_completo,
    CAST(Company AS STRING) AS empresa,
    CAST(Address AS STRING) AS endereco,
    CAST(City AS STRING) AS cidade,
    CAST(State AS STRING) AS uf,
    CAST(UPPER(Country) AS STRING) AS pais,
    CAST(PostalCode AS STRING) AS cep,
    CAST(Phone AS STRING) AS telefone,
    CAST(Fax AS STRING) AS fax,
    CAST(Email AS STRING) AS email,
    CAST(SupportRepId AS INT64) AS loja
FROM `{project_id}.camada_silver.Customer`
ORDER BY id_vendedor ASC
"""

# Executar a query para inserir os dados
try:
    query_job = client.query(sql)
    query_job.result()  # Aguarda a execução da query
    logging.info("Dados inseridos com sucesso na tabela.")
except Exception as e:
    logging.error(f"Erro ao inserir dados na tabela: {e}")

INFO:root:Tabela criada com sucesso com clustering e campo `id_vendedor` como REQUIRED.
INFO:root:Dados inseridos com sucesso na tabela.


In [6]:
# Nome do projeto, dataset e tabela
project_id = "sincere-actor-393814"
dataset_id = "camada_gold"
table_id = "dArtista"

# Definir o schema com `NOT NULL` e descrição
schema = [
    bigquery.SchemaField("id_artista", "INTEGER", mode="REQUIRED", description="Identificador único do artista"),
    bigquery.SchemaField("nome_artista", "STRING", mode="NULLABLE", description="Nome completo do artista"),
    bigquery.SchemaField("genero_musical", "STRING", mode="NULLABLE", description="Gênero musical do artista"),
    bigquery.SchemaField("titulo_disco", "STRING", mode="NULLABLE", description="Nome do disco"),
]

# Configurar a referência para a tabela com clustering
table_ref = client.dataset(dataset_id).table(table_id)
table = bigquery.Table(table_ref, schema=schema)
table.clustering_fields = ["id_artista"]  # Definir o campo de clustering

# Criar a tabela com o schema e clustering
try:
    table = client.create_table(table, exists_ok=True)
    logging.info("Tabela criada com sucesso com clustering e campo `id_artista` como REQUIRED.")
except Exception as e:
    logging.error(f"Erro ao criar a tabela: {e}")

# Inserir dados na tabela a partir da consulta SQL
sql = f"""
INSERT INTO `{project_id}.{dataset_id}.{table_id}`
SELECT 
    DISTINCT 
    A.ArtistId AS id_artista,
    A.Name AS nome_artista,
    CASE 
        -- Corrigir B.Name com base em A.Name
        WHEN A.Name LIKE '%Rock%' THEN 'Rock'
        WHEN A.Name LIKE '%Metallica%' THEN 'Rock'
        WHEN A.Name LIKE '%Kid Abelha%' THEN 'Rock'
        WHEN A.Name LIKE '%Barão Vermelho%' THEN 'Rock'
        WHEN A.Name LIKE '%Banda Black Rio%' THEN 'Rock'
        WHEN A.Name LIKE '%Azymuth%' THEN 'Rock'
        WHEN A.Name LIKE '%Ed Motta%' THEN 'Soul'
        -- Se B.Name estiver nulo ou vazio, preencher com "MPB"
        WHEN B.Name IS NULL OR TRIM(B.Name) = '' THEN 'MPB'
        -- Caso contrário, manter o valor original de B.Name
        ELSE B.Name
    END AS genero_musical,
    C.Title as titulo_disco
FROM `{project_id}.camada_silver.Artist` AS A
LEFT JOIN `{project_id}.camada_silver.Genre` AS B
    ON TRIM(CAST(A.ArtistId AS STRING)) = TRIM(CAST(B.GenreId AS STRING))
 LEFT JOIN `{project_id}.camada_silver.Album` AS C
 ON TRIM(CAST(A.ArtistId AS STRING)) = TRIM(CAST(C.ArtistId AS STRING))
ORDER BY id_artista ASC;
"""

# Executar a query para inserir os dados
try:
    query_job = client.query(sql)
    query_job.result()  # Aguarda a execução da query
    logging.info("Dados inseridos com sucesso na tabela.")
except Exception as e:
    logging.error(f"Erro ao inserir dados na tabela: {e}")

INFO:root:Tabela criada com sucesso com clustering e campo `id_artista` como REQUIRED.
INFO:root:Dados inseridos com sucesso na tabela.


In [12]:

# Nome do projeto, dataset e tabela
project_id = "sincere-actor-393814"
dataset_id = "camada_gold"
table_id = "fVendas_Music"

# Definir o schema com `NOT NULL` e descrição
schema = [
    bigquery.SchemaField("data_fatura", "DATE", mode="REQUIRED", description="Data da fatura"),
    bigquery.SchemaField("id_fatura", "INTEGER", mode="REQUIRED", description="Identificador único da fatura"),
    bigquery.SchemaField("id_artista", "INTEGER", mode="REQUIRED", description="Identificador único do artista"),
    bigquery.SchemaField("id_vendedor", "INTEGER", mode="REQUIRED", description="Identificador único do vendedor"),
    bigquery.SchemaField("id_rastreio", "INTEGER", mode="REQUIRED", description="Identificador único do rastreamento de música"),
    bigquery.SchemaField("quantidade", "INTEGER", mode="NULLABLE", description="Quantidade vendida na fatura"),
    bigquery.SchemaField("preco_unitario", "FLOAT", mode="NULLABLE", description="Preço unitário do item"),
    bigquery.SchemaField("total", "FLOAT", mode="NULLABLE", description="Valor total da fatura"),
    bigquery.SchemaField("endereco", "STRING", mode="NULLABLE", description="Endereço de cobrança"),
    bigquery.SchemaField("cidade", "STRING", mode="NULLABLE", description="Cidade de cobrança"),
    bigquery.SchemaField("estado", "STRING", mode="NULLABLE", description="Estado de cobrança"),
    bigquery.SchemaField("pais", "STRING", mode="NULLABLE", description="País de cobrança"),
    bigquery.SchemaField("cep", "STRING", mode="NULLABLE", description="Código postal de cobrança"),
    bigquery.SchemaField("id_media", "INTEGER", mode="REQUIRED", description="Identificador único do tipo de mídia"),
]

# Configurar a referência para a tabela com clustering
table_ref = client.dataset(dataset_id).table(table_id)
table = bigquery.Table(table_ref, schema=schema)

# Ajustar para 4 campos de clustering
table.clustering_fields = ["id_fatura", "id_vendedor", "id_artista", "id_rastreio"]  # Limitado a 4 campos

# Criar a tabela com o schema e clustering
try:
    table = client.create_table(table, exists_ok=True)
    logging.info("Tabela criada com sucesso com clustering e campos configurados.")
except Exception as e:
    logging.error(f"Erro ao criar a tabela: {e}")

# SQL para inserir dados na tabela
sql = f"""
SELECT
    CAST(PARSE_DATE('%Y-%m-%d', LEFT(I.InvoiceDate, 10))AS DATE) AS data_fatura,
    CAST(IL.InvoiceLineId AS INT64) AS id_fatura,
    COALESCE(CAST(A.ArtistId AS INT64), 0) AS id_artista,  -- Substituir NULL por 0
    CAST(I.CustomerId AS INT64) AS id_vendedor,
    CAST(M.MediaTypeId AS INT64) AS id_media,
    CAST(IL.TrackId AS INT64) AS id_rastreio,
    CAST(IL.Quantity AS INT64) AS quantidade,
    CAST(IL.UnitPrice AS FLOAT64) AS preco_unitario,
    CAST(I.Total AS FLOAT64) AS total,
    CAST(I.BillingAddress AS STRING) AS endereco,
    CAST(I.BillingCity AS STRING) AS cidade,
    CAST(I.BillingState AS STRING) AS estado,
    CAST(I.BillingCountry AS STRING) AS pais,
    CAST(I.BillingPostalCode AS STRING) AS cep
FROM `{project_id}.camada_silver.InvoiceLine` AS IL
LEFT JOIN `{project_id}.camada_silver.Invoice` AS I
    ON IL.InvoiceId = I.InvoiceId
LEFT JOIN `{project_id}.camada_silver.Artist` AS A
    ON I.InvoiceId = A.ArtistId
LEFT JOIN `{project_id}.camada_silver.Track` AS T
    ON IL.TrackId = T.TrackId
LEFT JOIN `{project_id}.camada_silver.MediaType` AS M
    ON T.MediaTypeId  = M.MediaTypeId
ORDER BY IL.InvoiceLineId ASC;
"""

# Executar a query para inserir os dados
try:
    query_job = client.query(sql)
    query_job.result()  # Aguarda a execução da query
    logging.info("Dados inseridos com sucesso na tabela.")
except Exception as e:
    logging.error(f"Erro ao inserir dados na tabela: {e}")



INFO:root:Tabela criada com sucesso com clustering e campos configurados.
INFO:root:Dados inseridos com sucesso na tabela.


In [23]:
# Nome do projeto, dataset e tabela
project_id = "sincere-actor-393814"
dataset_id = "camada_gold"
table_id = "fFaixas"

# Definir o schema com `NOT NULL` e descrição
schema = [
    bigquery.SchemaField("id_faixa", "INTEGER", mode="REQUIRED", description="Identificador único da faixa"),
    bigquery.SchemaField("id_artista", "INTEGER", mode="REQUIRED", description="Identificador único do artista"),
    bigquery.SchemaField("id_media", "INTEGER", mode="REQUIRED", description="Identificador único do tipo de mídia"),
    bigquery.SchemaField("faixa", "STRING", mode="NULLABLE", description="Nome da faixa"),
    bigquery.SchemaField("compositor", "STRING", mode="NULLABLE", description="Compositor da faixa"),
    bigquery.SchemaField("tempo_segundos", "FLOAT", mode="NULLABLE", description="Duração da faixa em segundos"),
    bigquery.SchemaField("bytes", "FLOAT", mode="NULLABLE", description="Tamanho do arquivo da faixa em bytes"),
    bigquery.SchemaField("preco_unitario", "STRING", mode="NULLABLE", description="Preço unitário da faixa formatado como texto"),
]

# Configurar a referência para a tabela com clustering
table_ref = client.dataset(dataset_id).table(table_id)
table = bigquery.Table(table_ref, schema=schema)
table.clustering_fields = ["id_faixa", "id_artista", "id_media"]  # Definir os campos de clustering

# Criar a tabela com o schema e clustering
try:
    table = client.create_table(table, exists_ok=True)
    logging.info("Tabela criada com sucesso com clustering e campos configurados.")
except Exception as e:
    logging.error(f"Erro ao criar a tabela: {e}")

# Inserir dados na tabela a partir da consulta SQL
sql = f"""
INSERT INTO `{project_id}.{dataset_id}.{table_id}`
SELECT 
    CAST(T.TrackId AS INT64) AS id_faixa,
    CAST(A.ArtistId AS INT64) AS id_artista,
    CAST(T.MediaTypeId AS INT64) AS id_media,
    CAST(T.Name AS STRING) AS faixa,
    CAST(T.Composer AS STRING) AS compositor,
    CAST(T.Milliseconds / 1000 AS FLOAT64) AS tempo_segundos,  -- Convertendo milissegundos para segundos
    CAST(T.Bytes AS FLOAT64) AS bytes,
    REPLACE(CAST(T.UnitPrice AS STRING), ".", ",") AS preco_unitario  -- Converter para string e substituir ponto por vírgula
FROM `{project_id}.camada_silver.Track` AS T
LEFT JOIN `{project_id}.camada_silver.Album` AS AL
    ON T.AlbumId = AL.AlbumId  -- Junta Track com Album
LEFT JOIN `{project_id}.camada_silver.Artist` AS A
    ON AL.ArtistId = A.ArtistId  -- Junta Album com Artist
ORDER BY T.TrackId ASC;
"""

# Executar a query para inserir os dados
try:
    query_job = client.query(sql)
    query_job.result()  # Aguarda a execução da query
    logging.info("Dados inseridos com sucesso na tabela.")
except Exception as e:
    logging.error(f"Erro ao inserir dados na tabela: {e}")


INFO:root:Tabela criada com sucesso com clustering e campos configurados.
INFO:root:Dados inseridos com sucesso na tabela.


In [24]:
# Nome do projeto, dataset e tabela
project_id = "sincere-actor-393814"
dataset_id = "camada_gold"
table_id = "dMidia"

# Definir o schema com `NOT NULL` e descrição
schema = [
    bigquery.SchemaField("id_media", "INTEGER", mode="REQUIRED", description="Identificador único do tipo de mídia"),
    bigquery.SchemaField("tipo_midia", "STRING", mode="NULLABLE", description="Descrição do tipo de mídia"),
]

# Configurar a referência para a tabela com clustering
table_ref = client.dataset(dataset_id).table(table_id)
table = bigquery.Table(table_ref, schema=schema)
table.clustering_fields = ["id_media"]  # Definir os campos de clustering

# Criar a tabela com o schema e clustering
try:
    table = client.create_table(table, exists_ok=True)
    logging.info("Tabela criada com sucesso com clustering e campos configurados.")
except Exception as e:
    logging.error(f"Erro ao criar a tabela: {e}")

# Inserir dados na tabela a partir da consulta SQL
sql = f"""
INSERT INTO `{project_id}.{dataset_id}.{table_id}`
SELECT 
    CAST(MediaTypeId AS INT64) AS id_media,
    CAST(Name AS STRING) AS tipo_midia
FROM `{project_id}.camada_silver.MediaType`
"""

# Executar a query para inserir os dados
try:
    query_job = client.query(sql)
    query_job.result()  # Aguarda a execução da query
    logging.info("Dados inseridos com sucesso na tabela.")
except Exception as e:
    logging.error(f"Erro ao inserir dados na tabela: {e}")


INFO:root:Tabela criada com sucesso com clustering e campos configurados.
INFO:root:Dados inseridos com sucesso na tabela.
